### *IPCC SR15 scenario assessment*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of global CO2 and Kyoto emissions, (BE)CCS <br /> and year of net-zero

This notebook computes indicators and diagnostics of emissions pathways, the use of carbon capture and sequestration, and the timing of net-zero of different emissions categories 
in the IPCC's _"Special Report on Global Warming of 1.5°C"_. The notebook generates the data for **Table 2.4** in the Special Report.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

To reduce potential bias by many scenarios from the same modelling framework, 13 scenarios submitted by the 'AIM' model are excluded from the assessment underpinning this statement (cf. SPM Statement C1).

Also, note that we apply the filter by relevant years *after computing the year of netzero*.

In [ ]:
cats.remove('Above 2C')

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
filter_args_aim = dict(model='AIM*',
                       scenario=['SFCM*_1p5Degree', 'EMF33_Med2C_nofuel', 'EMF33_Med2C_none'],
                       keep=False)

In [ ]:
df = (
    sr1p5
    .filter(kyoto_ghg_2010='in range', category=cats)
    .filter(**filter_args_aim)
)

## Initialize a `pyam.Statistics` instance

In [ ]:
stats = pyam.Statistics(df=df, groupby={'category': cats}, rows=True)

In [ ]:
years = [2030, 2050, 2100]
compare_years = [(2010, 2030), (2020, 2030), (2030, 2050)]

## Function to compute the year of netzero and add growth statistics to the summary

In [ ]:
def year_of_net_zero(data, years, threshold):
    prev_val = 0
    prev_yr = np.nan

    for yr, val in zip(years, data):
        if np.isnan(val):
            continue
        
        if val < threshold:
            x = (val - prev_val) / (yr - prev_yr) # absolute change per year
            return prev_yr + int((threshold - prev_val) / x) + 1 # add one because int() rounds down
        
        prev_val = val
        prev_yr = yr
    return np.inf

In [ ]:
header='Annual emissions/sequestration (GtCO2)'
header_change='Absolute annual change (GtCO2)'
header_zero='Timing of global zero'

statistics_settings = dict(
    header=header,
    header_change=header_change,
    header_zero= header_zero,
    years=years,
    compare_years=[(2010, 2030), (2020, 2030), (2030, 2050)],
)

In [ ]:
def add_statistics(data, row, years, compare_years,
                  header, header_change, header_zero, add_netzero=False):
    stats.add(data[years], header=header, row=row)
    for i, j in compare_years:
        abs_ann_change = (data[j] - data[i]) / (j - i)
        stats.add(abs_ann_change, header=header_change, row=row,
                       subheader='{}-{}'.format(i,j))
    if add_netzero:
        netzero = data.apply(year_of_net_zero, years=data.columns, threshold=0, axis=1)
        stats.add(netzero, header=header_zero, row=row, subheader='year')

## Get timeseries of total CO2 emissions

In [ ]:
co2 = (
    df.filter(variable='Emissions|CO2')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)

In [ ]:
add_statistics(co2, 'Total CO2 (net)', **statistics_settings, add_netzero=True)

In [ ]:
co2_gross_seq_variables = [
    'Carbon Sequestration|CCS|Biomass',
    'Carbon Sequestration|Land Use',
    'Carbon Sequestration|Direct Air Capture',
    'Carbon Sequestration|Enhanced Weathering'
]
agg_sequestration = (
    df.filter(variable=co2_gross_seq_variables)
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)
agg_sequestration = agg_sequestration.groupby(pyam.META_IDX).sum()

In [ ]:
co2_ene_ind = (
    df.filter(variable='Emissions|CO2|Energy and Industrial Processes')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)
co2_ene_ind.index = co2_ene_ind.index.droplevel([2, 3, 4])

In [ ]:
co2_ene_ind_gross = (co2_ene_ind + agg_sequestration).combine_first(co2_ene_ind)

In [ ]:
add_statistics(co2_ene_ind_gross, 'CO2 from fossil fuels and industry (gross)', **statistics_settings)

In [ ]:
add_statistics(co2_ene_ind, 'CO2 from fossil fuels and industry (net)', **statistics_settings)

In [ ]:
co2_afolu = (
    df.filter(variable='Emissions|CO2|AFOLU')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)

In [ ]:
add_statistics(co2_afolu, 'CO2 from AFOLU', **statistics_settings)

## CCS from bioenergy

In [ ]:
ccs_bio = (
    df.filter(variable='Carbon Sequestration|CCS|Biomass')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)

In [ ]:
add_statistics(ccs_bio, 'Bioenergy combined with carbon capture and storage (BECCS)',**statistics_settings)

## Total greenhouse gases according to the Kyoto protocol

In [ ]:
ghg = (
    df.filter(variable='Emissions|Kyoto Gases (AR4-GWP100)')
    .convert_unit({'Mt CO2-equiv/yr': ('Gt CO2-equiv/yr', 0.001)})
    .timeseries()
)

In [ ]:
add_statistics(ghg, 'Kyoto GHG (AR4, GtCO2e)', **statistics_settings, add_netzero=True)

## Display and export summary statistics to `xlsx`

Note that in Table 2.4 as printed in the pre-release of the Special Report following the approval plenary, the full range if shown for any cells with less than 7 scenarios, and interquartile ranges are shown otherwise.
This formatting was implemented manually ex-post, as it is currently not supported by the `pyam.Statistics` module.

In [ ]:
summary = stats.summarize(center='median', interquartile=True)
summary

In [ ]:
summary.to_excel('output/table_2.4_emission_statistics.xlsx')